In [1]:
import pandas as pd
import datetime

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

from collections import Counter

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

### Consider the first attribute

In [2]:
input_price = pd.read_excel("daily_trading_info_2014-2018.xlsx", sheet_name = None)

In [3]:
stock_index = []
stock_index = list(input_price.keys())[0:]

In [4]:
results = pd.read_excel("strategy_results_train_copy.xlsx", sheet_name = "strategy_results_train")

In [5]:
import copy

In [6]:
dataset = copy.deepcopy(results)

In [7]:
len(dataset)

2000

In [8]:
from talib import abstract

In [9]:
def ma(input_price, trade_company, trade_date_datetime):
    df = input_price[trade_company]
    lower_index = 0
    for index in range(0, df.shape[0]):
        if(df["date"][index].year == trade_date_datetime.year)\
        and(df["date"][index].month <= trade_date_datetime.month):
            lower_index = index

    inputs = {
        'open': df["open"].iloc[:lower_index],
        'high': df["high"].iloc[:lower_index],
        'low':  df["low"].iloc[:lower_index],
        'close': df["close"].iloc[:lower_index],
        'volume': df["volume"].iloc[:lower_index]
    }
    # uses close prices (default)
    output = abstract.SMA(inputs, timeperiod=5)
    return output[-1]

In [10]:
temp = []
for i in range(0, dataset.shape[0]):
    if(i%1000==0):
        print(i, datetime.datetime.now())
    trade_company = dataset.iloc[i][0].split("_")[0]
    trade_date    = dataset.iloc[i][0].split("_")[1]
    trade_date_datetime = datetime.date(int(trade_date[:4]), int(trade_date[4:]), 1) - datetime.timedelta(days=1)
    temp.append(ma(input_price, trade_company, trade_date_datetime))
#     print(i, trade_company, trade_date, trade_date_datetime)

0 2020-05-25 15:35:40.666001
1000 2020-05-25 15:35:55.213024


In [11]:
len(temp)

2000

In [12]:
pd.DataFrame(temp, columns=["ma"]).head()
temp = pd.DataFrame(temp, columns=["ma"])

In [13]:
temp.head()

,ma
0,21.291999
1,23.492000
2,22.320000
3,26.378000
4,32.888000


In [14]:
dataset.shape

(2000, 2)

In [15]:
dataset.head()

,id,label
0,SHSE.600085_201501,0
1,SHSE.600085_201502,1
2,SHSE.600085_201503,1
3,SHSE.600085_201504,1
4,SHSE.600085_201505,1


In [16]:
result = pd.concat([dataset, pd.DataFrame(temp).reindex(dataset.index)], axis=1)
result.head()

,id,label,ma
0,SHSE.600085_201501,0,21.291999
1,SHSE.600085_201502,1,23.492000
2,SHSE.600085_201503,1,22.320000
3,SHSE.600085_201504,1,26.378000
4,SHSE.600085_201505,1,32.888000


In [17]:
result.to_excel("train_output.xlsx", index = False)  

### Consider more features

In [18]:
results = pd.read_excel("train_output.xlsx", sheet_name = "Sheet1")

In [19]:
results.shape

(2000, 3)

In [20]:
results.head()

,id,label,ma
0,SHSE.600085_201501,0,21.291999
1,SHSE.600085_201502,1,23.492000
2,SHSE.600085_201503,1,22.320000
3,SHSE.600085_201504,1,26.378000
4,SHSE.600085_201505,1,32.888000


In [21]:
import copy

In [22]:
dataset = copy.deepcopy(results)

In [23]:
len(dataset)

2000

##### START: Caculating the five-day average close price of SHSE.600085 at 2015-01-31

In [24]:
def ma_with_exp(input_price, trade_company, trade_date_datetime):
    df = input_price[trade_company]
    lower_index = 0
    for index in range(0, df.shape[0]):
        if(df["date"][index].year == trade_date_datetime.year)\
        and(df["date"][index].month <= trade_date_datetime.month):
            lower_index = index

    inputs = {
        'open': df["open"].iloc[:lower_index],
        'high': df["high"].iloc[:lower_index],
        'low':  df["low"].iloc[:lower_index],
        'close': df["close"].iloc[:lower_index],
        'volume': df["volume"].iloc[:lower_index]
    }
    # uses close prices (default)
    output = abstract.EMA(inputs, 5)
    return output[-1]

In [25]:
temp = []
for i in range(0, dataset.shape[0]):
    if(i%1000==0):
        print(i, datetime.datetime.now())
    trade_company = dataset.iloc[i][0].split("_")[0]
    trade_date    = dataset.iloc[i][0].split("_")[1]
    trade_date_datetime = datetime.date(int(trade_date[:4]), int(trade_date[4:]), 1) - datetime.timedelta(days=1)
    temp.append(ma_with_exp(input_price, trade_company, trade_date_datetime))
#     print(i, trade_company, trade_date, trade_date_datetime)

0 2020-05-25 15:36:08.911300
1000 2020-05-25 15:36:22.989624


In [26]:
len(temp)

2000

In [27]:
pd.DataFrame(temp, columns=["ma_with_exp"]).head()
temp = pd.DataFrame(temp, columns=["ma_with_exp"])

In [28]:
dataset.shape

(2000, 3)

In [29]:
result = pd.concat([dataset, pd.DataFrame(temp).reindex(dataset.index)], axis=1)
result.head()

,id,label,ma,ma_with_exp
0,SHSE.600085_201501,0,21.291999,21.437717
1,SHSE.600085_201502,1,23.492000,23.357240
2,SHSE.600085_201503,1,22.320000,22.334699
3,SHSE.600085_201504,1,26.378000,25.994325
4,SHSE.600085_201505,1,32.888000,32.987757


In [30]:
result.to_excel("train_output.xlsx", index = False)  

### Prediction

In [31]:
results = pd.read_excel("train_output.xlsx", sheet_name = "Sheet1")

In [32]:
results.head()

,id,label,ma,ma_with_exp
0,SHSE.600085_201501,0,21.291999,21.437717
1,SHSE.600085_201502,1,23.492000,23.357240
2,SHSE.600085_201503,1,22.320000,22.334699
3,SHSE.600085_201504,1,26.378000,25.994325
4,SHSE.600085_201505,1,32.888000,32.987757


In [33]:
results.isnull().values.any()

True

In [34]:
results.dropna(axis=0)

,id,label,ma,ma_with_exp
0,SHSE.600085_201501,0,21.291999,21.437717
1,SHSE.600085_201502,1,23.492000,23.357240
2,SHSE.600085_201503,1,22.320000,22.334699
3,SHSE.600085_201504,1,26.378000,25.994325
4,SHSE.600085_201505,1,32.888000,32.987757
...,...,...,...,...
1995,SHSE.601006_201612,0,7.152000,7.183480
1996,SHSE.601006_201701,0,7.048000,7.035677
1997,SHSE.601006_201702,0,7.164000,7.156839
1998,SHSE.601006_201703,1,7.028000,7.011566


In [35]:
results = results.dropna(axis=0)

In [36]:
results.isnull().values.any()

False

In [37]:
y = pd.DataFrame(results["label"])
X = pd.DataFrame(results.iloc[:, 2:4])

In [38]:
y.head()

,label
0,0
1,1
2,1
3,1
4,1


In [39]:
X.head()

,ma,ma_with_exp
0,21.291999,21.437717
1,23.492000,23.357240
2,22.320000,22.334699
3,26.378000,25.994325
4,32.888000,32.987757


In [40]:
len(X)

1998

In [41]:
clf = LogisticRegression()
clf.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

#### 開始處理 test set

In [42]:
test_input_price = pd.read_excel("daily_trading_info_2019-2020.xlsx", sheet_name = None)

In [43]:
test_stock_index = []
test_stock_index = list(test_input_price.keys())[0:]

In [44]:
test_results = pd.read_csv("sampleSubmission.csv")

In [45]:
test_results.head()

,id,label
0,SHSE.600085_201901,1
1,SHSE.600085_201902,1
2,SHSE.600085_201903,1
3,SHSE.600085_201904,1
4,SHSE.600085_201905,1


In [46]:
import copy

In [47]:
test_dataset = copy.deepcopy(test_results)

In [48]:
def ma2(test_input_price, input_price, trade_company,trade_date_datetime):
    df = test_input_price[trade_company]
    lower_index = 0
    for index in range(0, df.shape[0]):
        if(df["date"][index].year == trade_date_datetime.year)\
        and(df["date"][index].month <= trade_date_datetime.month):
            lower_index = index

    if(lower_index == 0):
        output = ma(input_price, trade_company,trade_date_datetime)
    else:       
        inputs = {
            'open': df["open"].iloc[:lower_index],
            'high': df["high"].iloc[:lower_index],
            'low':  df["low"].iloc[:lower_index],
            'close': df["close"].iloc[:lower_index],
            'volume': df["volume"].iloc[:lower_index]
        }
        # uses close prices (default)
        output = abstract.SMA(inputs, timeperiod=5)
        output = output[-1]
    return output

In [49]:
temp = []
for i in range(0, test_dataset.shape[0]):
    if(i%1000==0):
        print(i, datetime.datetime.now())
    trade_company = test_dataset.iloc[i][0].split("_")[0]
    trade_date    = test_dataset.iloc[i][0].split("_")[1]
    trade_date_datetime = datetime.date(int(trade_date[:4]), int(trade_date[4:]), 1) - datetime.timedelta(days=1)
    temp.append(ma2(test_input_price, input_price, trade_company,trade_date_datetime))
#     print(i, trade_company, trade_date)

0 2020-05-25 15:36:55.371684
1000 2020-05-25 15:37:01.895975
2000 2020-05-25 15:37:08.226278
3000 2020-05-25 15:37:14.530248
4000 2020-05-25 15:37:20.556366
5000 2020-05-25 15:37:26.738736
6000 2020-05-25 15:37:33.202090
7000 2020-05-25 15:37:39.348935
8000 2020-05-25 15:37:45.539418
9000 2020-05-25 15:37:51.811003
10000 2020-05-25 15:37:58.332424
11000 2020-05-25 15:38:04.524227


In [50]:
len(temp)

11363

In [51]:
test_dataset.shape[0]

11363

In [52]:
temp = pd.DataFrame(temp, columns=["ma"])
test_result = pd.concat([test_dataset, temp], axis=1)
test_result.head()

,id,label,ma
0,SHSE.600085_201901,1,27.124
1,SHSE.600085_201902,1,26.624
2,SHSE.600085_201903,1,28.890
3,SHSE.600085_201904,1,29.560
4,SHSE.600085_201905,1,31.514


In [53]:
test_result.to_excel("test_output.xlsx", index = None)  

In [54]:
results = pd.read_excel("test_output.xlsx", sheet_name = "Sheet1")

In [55]:
results.shape

(11363, 3)

In [56]:
results.head()

,id,label,ma
0,SHSE.600085_201901,1,27.124
1,SHSE.600085_201902,1,26.624
2,SHSE.600085_201903,1,28.890
3,SHSE.600085_201904,1,29.560
4,SHSE.600085_201905,1,31.514


In [57]:
import copy

In [58]:
dataset = copy.deepcopy(results)

In [59]:
len(dataset)

11363

In [60]:
def ma_with_exp2(test_input_price, input_price, trade_company, trade_date_datetime):
    df = test_input_price[trade_company]
    lower_index = 0
    for index in range(0, df.shape[0]):
        if(df["date"][index].year == trade_date_datetime.year)\
        and(df["date"][index].month <= trade_date_datetime.month):
            lower_index = index
            
    if(lower_index == 0):
        output = ma_with_exp(input_price, trade_company,trade_date_datetime)
    else:
        inputs = {
            'open': df["open"].iloc[:lower_index],
            'high': df["high"].iloc[:lower_index],
            'low':  df["low"].iloc[:lower_index],
            'close': df["close"].iloc[:lower_index],
            'volume': df["volume"].iloc[:lower_index]
        }
        # uses close prices (default)
        output = abstract.EMA(inputs, 5)
        output = output[-1]
    return output

In [61]:
temp = []
for i in range(0, dataset.shape[0]):
    if(i%1000==0):
        print(i, datetime.datetime.now())
    trade_company = dataset.iloc[i][0].split("_")[0]
    trade_date    = dataset.iloc[i][0].split("_")[1]
    trade_date_datetime = datetime.date(int(trade_date[:4]), int(trade_date[4:]), 1) - datetime.timedelta(days=1)
    temp.append(ma_with_exp2(test_input_price, input_price, trade_company, trade_date_datetime))
#     print(i, trade_company, trade_date, trade_date_datetime)

0 2020-05-25 15:38:07.861866
1000 2020-05-25 15:38:14.190479
2000 2020-05-25 15:38:20.504395
3000 2020-05-25 15:38:26.696523
4000 2020-05-25 15:38:32.908268
5000 2020-05-25 15:38:39.524817
6000 2020-05-25 15:38:45.783493
7000 2020-05-25 15:38:52.197952
8000 2020-05-25 15:38:58.587164
9000 2020-05-25 15:39:04.729032
10000 2020-05-25 15:39:11.027432
11000 2020-05-25 15:39:17.284000


In [62]:
len(temp)

11363

In [63]:
pd.DataFrame(temp, columns=["ma_with_exp"]).head()
temp = pd.DataFrame(temp, columns=["ma_with_exp"])

In [64]:
dataset.shape

(11363, 3)

In [65]:
result = pd.concat([dataset, pd.DataFrame(temp).reindex(dataset.index)], axis=1)
result.head()

,id,label,ma,ma_with_exp
0,SHSE.600085_201901,1,27.124,27.301295
1,SHSE.600085_201902,1,26.624,26.610847
2,SHSE.600085_201903,1,28.890,29.030110
3,SHSE.600085_201904,1,29.560,29.582097
4,SHSE.600085_201905,1,31.514,31.423752


In [66]:
result.to_excel("test_output.xlsx", index = False)  

In [67]:
test_results = pd.read_excel("test_output.xlsx", sheet_name = "Sheet1")

In [68]:
test_results.head()

,id,label,ma,ma_with_exp
0,SHSE.600085_201901,1,27.124,27.301295
1,SHSE.600085_201902,1,26.624,26.610847
2,SHSE.600085_201903,1,28.890,29.030110
3,SHSE.600085_201904,1,29.560,29.582097
4,SHSE.600085_201905,1,31.514,31.423752


In [69]:
test_results.isnull().any()

id             False
label          False
ma              True
ma_with_exp     True
dtype: bool

In [70]:
test_results.isnull().any().sum()

2

In [71]:
test_results[test_results["ma"].isnull()]

,id,label,ma,ma_with_exp
10992,SZSE.002946_201902,0,NaN,NaN


In [72]:
test_results[test_results["ma"].isnull()].id

10992    SZSE.002946_201902
Name: id, dtype: object

In [73]:
test_input_price["SZSE.002946"]

,Unnamed: 0,date,open,high,low,close,volume
0,0,2019-01-25,6.54,7.85,6.54,7.85,103502
1,1,2019-01-28,8.64,8.64,8.64,8.64,65762
2,2,2019-01-29,9.50,9.50,9.50,9.50,230822
3,3,2019-01-30,10.45,10.45,10.45,10.45,304355
4,4,2019-01-31,11.50,11.50,11.50,11.50,640811
...,...,...,...,...,...,...,...
280,280,2020-03-25,10.70,11.57,10.50,11.22,10236404
281,281,2020-03-26,11.00,12.00,10.98,11.49,10871703
282,282,2020-03-27,11.48,11.86,11.48,11.57,7054557
283,283,2020-03-30,11.62,12.22,11.56,11.90,8474864


In [74]:
test_results = test_results.dropna(axis = 0)

In [75]:
test_results.iloc[:, 2:4].head()

,ma,ma_with_exp
0,27.124,27.301295
1,26.624,26.610847
2,28.890,29.030110
3,29.560,29.582097
4,31.514,31.423752


In [76]:
test_X = test_results.iloc[:, 2:4]

In [77]:
test_X

,ma,ma_with_exp
0,27.124000,27.301295
1,26.624000,26.610847
2,28.890000,29.030110
3,29.560000,29.582097
4,31.514000,31.423752
...,...,...
11358,86.210001,85.900316
11359,77.744000,78.256732
11360,70.966000,71.456738
11361,81.862001,82.689670


In [78]:
clf.predict(test_X)

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [79]:
pd.DataFrame(clf.predict(test_X), columns= ["label"]).to_excel("prediction.xlsx")  

##### END: Caculating the five-day average close price of SHSE.600085 at 2015-01-31